In [50]:
import math
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
def get_current_parameter(metrics, parameter):
    try:
        value = current_config[metrics][parameter]
    except:
        value = 0
    return value

def get_new_parameter(metrics, parameter):
    try:
        value = new_config[metrics][parameter]
    except:
        value = 0
    return value


def read_json(source: str, directory: str) -> dict:
    if source == "directory":
        with open(directory) as json_file:
            config = json.load(json_file)
            json_file.close()
    elif source == "url":
        config = requests.get(directory).json()
    elif source == "topads_api":
        r = requests.get(directory).json()
        config = r["data"][0]["criterion"]
    else:
        raise InvalidSourceError(f"source {source} is not supported")
    return config

def scurve(steepness, midpoint, startpoint, limit, threshold, value):
    if threshold >= limit:
        return 1 / (1 + np.exp(-steepness * (value - midpoint)))
    else:
        return startpoint

# Load Dataset

In [52]:
# Import dataset
df = pd.read_csv('task-4-scoring.csv')
df.head()

,ID,Join Date,Date of Birth,Gender,City,GMV (in scale),Count of Invoice,Count of Customer,%GMV of Cigarettes,Total Voucher/Discount (in scale),Count of Active Days,Count of Cancelled Orders,Profit (in scale),Commission (in scale),Count of SKU,Count of Category
0,1085,2021-03-18,1988-09-04,L,Surabaya,20760,6,4,0.91,0,6,0,104,40,9,5
1,11424,2020-10-06,1989-03-26,P,Malang,10335,16,4,0.09,55,14,0,236,158,57,10
2,12631,2020-10-01,1995-10-07,L,Sidoarjo,58894,61,26,0.28,132,22,4,776,625,90,11
3,1281,2020-11-27,1986-09-22,P,Surabaya,550416,320,65,0.62,623,30,6,6640,3188,214,13
4,15076,2020-09-28,1997-10-24,L,Surabaya,34808,41,20,0.17,72,19,0,502,616,63,10


In [53]:
# Columns for which S-curve and Relu need to be calculated
features = ["Count of Customer", "Count of Invoice", "%GMV of Cigarettes", "GMV (in scale)"]
threshold = {
    "Count of Customer": {
        "metrics": "Count of Customer"
    },
    "Count of Invoice": {
        "metrics": "Count of Invoice"
    },
    "%GMV of Cigarettes": {
        "metrics": "%GMV of Cigarettes"
    },
    "GMV (in scale)": {
        "metrics": "GMV (in scale)"
    }
}

# Finding Current Distribution

In [54]:
adjusted_data = {
    'percentile': [0, 10, 25, 50, 75, 90, 95, 99, 100],
    'GMV (in scale)': [df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.1),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.25),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.5),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.75),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.9),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.95),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.99),
    df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(1)],
    'Count of Customer': [df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.1)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.25)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.5)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.75)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.9)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.95)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.99)*100,
    df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(1)*100],
    'Count of Invoice': [df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.1)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.25)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.5)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.75)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.9)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.95)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.99)*100,
    df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(1)*100],
    '%GMV of Cigarettes': [df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.1)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.25)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.5)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.75)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.9)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.95)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.99)*100,
    df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(1)*100]
}

In [55]:
df_adjusted_data = pd.DataFrame(adjusted_data)
df_adjusted_data

,percentile,GMV (in scale),Count of Customer,Count of Invoice,%GMV of Cigarettes
0,0,65.00,100.0,100.0,2.00
1,10,2000.30,100.0,370.0,8.50
2,25,6482.25,375.0,700.0,16.75
3,50,19210.00,850.0,2000.0,43.00
4,75,97764.75,2425.0,5800.0,75.00
5,90,315898.20,6500.0,16490.0,87.50
6,95,436247.50,8365.0,28095.0,92.25
7,99,706761.75,20227.0,102059.0,100.00
8,100,1024191.00,28200.0,244300.0,100.00


In [56]:
# Determine steepness
steepness_GMV = math.log(3) / (df["GMV (in scale)"].quantile(0.75) - df["GMV (in scale)"].quantile(0.50))
steepness_Count_of_Customer = math.log(3) / (df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.75) - df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.50)) 
steepness_Count_of_Invoice = math.log(3) / (df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.75) - df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.50))
steepness_GMV_of_Cigarettes = math.log(3) / (df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.75) - df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.50))

# Determine midpoint
midpoint_GMV = df["GMV (in scale)"].quantile(0.50)
midpoint_Count_of_Customer = df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.50)
midpoint_Count_of_Invoice = df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.50)
midpoint_GMV_of_Cigarettes = df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.50)

# Determine limit
limit_GMV  = 0
limit_Count_of_Customer = 0
limit_Count_of_Invoice = 0
limit_GMV_of_Cigarettes = 0

# Determine startpoint
startpoint_GMV = 1 / (1 + math.exp(-steepness_GMV * (df["GMV (in scale)"].quantile(0.25) - midpoint_GMV)))
startpoint_Count_of_Customer = 1 / (1 + math.exp(-steepness_Count_of_Customer * (df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.25) - midpoint_Count_of_Customer)))
startpoint_Count_of_Invoice = 1 / (1 + math.exp(-steepness_Count_of_Invoice * (df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.25) - midpoint_Count_of_Invoice)))
startpoint_GMV_of_Cigarettes = 1 / (1 + math.exp(-steepness_GMV_of_Cigarettes * (df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.25) - midpoint_GMV_of_Cigarettes)))


# Determine denom offset
denom_offset_GMV = 1 / df.loc[df['GMV (in scale)'] > 0, 'GMV (in scale)'].quantile(0.50)
denom_offset_Count_of_Customer = 1 / df.loc[df['Count of Customer'] > 0, 'Count of Customer'].quantile(0.50)
denom_offset_Count_of_Invoice = 1 / df.loc[df['Count of Invoice'] > 0, 'Count of Invoice'].quantile(0.50)
denom_offset_GMV_of_Cigarettes = 1 / df.loc[df['%GMV of Cigarettes'] > 0, '%GMV of Cigarettes'].quantile(0.50)

In [57]:
# Current Configuration
# current_config = read_json("topads_api", "http://10.41.67.249:30210/v2/formulas?name=product_browse_score")
current_config ={
    "GMV (in scale)": {
        "normalizer_type": "scurve",
        "scurve_steepness": steepness_GMV,
        "scurve_midpoint": midpoint_GMV,
        "scurve_limit": limit_GMV,
        "scurve_startpoint": startpoint_GMV,
    },
    "Count of Customer": {
        "normalizer_type": "scurve",
        "scurve_steepness": steepness_Count_of_Customer,
        "scurve_midpoint": midpoint_Count_of_Customer,
        "scurve_limit": limit_Count_of_Customer,
        "scurve_startpoint": startpoint_Count_of_Customer,
    },
    "%GMV of Cigarettes": {
        "normalizer_type": "scurve",
        "scurve_steepness": steepness_GMV_of_Cigarettes,
        "scurve_midpoint": midpoint_GMV_of_Cigarettes,
        "scurve_limit": limit_GMV_of_Cigarettes,
        "scurve_startpoint": startpoint_GMV_of_Cigarettes,
    },
    "Count of Invoice": {
        "normalizer_type": "scurve",
        "scurve_steepness": steepness_Count_of_Invoice,
        "scurve_midpoint": midpoint_Count_of_Invoice,
        "scurve_limit": limit_Count_of_Invoice,
        "scurve_startpoint": startpoint_Count_of_Invoice,
    }
}

In [58]:
# Calculate S-curve and Relu for each metrics using current config
for feature in features:
    if get_current_parameter(feature, "normalizer_type") == 'scurve':
        print("calculate s-curve " + feature + " using current config")
        df[f"f_current_{feature}"] = df.apply(lambda row: scurve(get_current_parameter(feature, "scurve_steepness"), get_current_parameter(feature, "scurve_midpoint"), get_current_parameter(feature, "scurve_startpoint"), get_current_parameter(feature, "scurve_limit"), row[threshold[feature]["metrics"]], row[feature]), axis=1)
    elif get_current_parameter(feature, "normalizer_type") == 'relu':
        print("calculate relu " + feature + " using current config")
        df[f"f_current_{feature}"] = df.apply(lambda row: relu(get_current_parameter(feature, "relu_balancer"), get_current_parameter(feature, "relu_penalty"), get_current_parameter(feature, "relu_slop"), get_current_parameter(feature, "relu_limit"), row[threshold[feature]["metrics"]], row[feature]), axis=1)

calculate s-curve Count of Customer using current config
calculate s-curve Count of Invoice using current config
calculate s-curve %GMV of Cigarettes using current config
calculate s-curve GMV (in scale) using current config


In [59]:
df.head()

,ID,Join Date,Date of Birth,Gender,City,GMV (in scale),Count of Invoice,Count of Customer,%GMV of Cigarettes,Total Voucher/Discount (in scale),Count of Active Days,Count of Cancelled Orders,Profit (in scale),Commission (in scale),Count of SKU,Count of Category,f_current_Count of Customer,f_current_Count of Invoice,f_current_%GMV of Cigarettes,f_current_GMV (in scale)
0,1085,2021-03-18,1988-09-04,L,Surabaya,20760,6,4,0.91,0,6,0,104,40,9,5,0.422166,0.400171,0.838610,0.505419
1,11424,2020-10-06,1989-03-26,P,Malang,10335,16,4,0.09,55,14,0,236,158,57,10,0.422166,0.471121,0.237348,0.469010
2,12631,2020-10-01,1995-10-07,L,Sidoarjo,58894,61,26,0.28,132,22,4,776,625,90,11,0.772183,0.765907,0.374028,0.635293
3,1281,2020-11-27,1986-09-22,P,Surabaya,550416,320,65,0.62,623,30,6,6640,3188,214,13,0.980943,0.999829,0.657529,0.999407
4,15076,2020-09-28,1997-10-24,L,Surabaya,34808,41,20,0.17,72,19,0,502,616,63,10,0.690437,0.647285,0.290569,0.554320


# Calculate Scoring

In [60]:
df.loc[:, "score"] = 25*df["f_current_Count of Customer"] + 25*df["f_current_Count of Invoice"] + 10*df[ "f_current_%GMV of Cigarettes"] + 40*df[ "f_current_GMV (in scale)"]

In [61]:
df.loc[:, 'rank'] = df.loc[:, 'score'].rank(method='first', ascending=False)
df

,ID,Join Date,Date of Birth,Gender,City,GMV (in scale),Count of Invoice,Count of Customer,%GMV of Cigarettes,Total Voucher/Discount (in scale),...,Profit (in scale),Commission (in scale),Count of SKU,Count of Category,f_current_Count of Customer,f_current_Count of Invoice,f_current_%GMV of Cigarettes,f_current_GMV (in scale),score,rank
0,1085,2021-03-18,1988-09-04,L,Surabaya,20760,6,4,0.91,0,...,104,40,9,5,0.422166,0.400171,0.838610,0.505419,49.161281,37.0
1,11424,2020-10-06,1989-03-26,P,Malang,10335,16,4,0.09,55,...,236,158,57,10,0.422166,0.471121,0.237348,0.469010,43.466049,54.0
2,12631,2020-10-01,1995-10-07,L,Sidoarjo,58894,61,26,0.28,132,...,776,625,90,11,0.772183,0.765907,0.374028,0.635293,67.604277,19.0
3,1281,2020-11-27,1986-09-22,P,Surabaya,550416,320,65,0.62,623,...,6640,3188,214,13,0.980943,0.999829,0.657529,0.999407,96.070839,4.0
4,15076,2020-09-28,1997-10-24,L,Surabaya,34808,41,20,0.17,72,...,502,616,63,10,0.690437,0.647285,0.290569,0.554320,58.521557,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,6323,2021-03-23,1995-12-20,L,Sidoarjo,2121,5,5,0.00,0,...,30,6,9,3,0.439267,0.393252,0.185993,0.440534,40.294276,61.0
64,6413,2020-10-01,1986-03-15,P,Surabaya,6503,8,4,0.00,0,...,92,33,30,5,0.422166,0.414127,0.185993,0.455689,40.994797,57.0
65,7155,2020-10-02,1985-09-15,P,Sidoarjo,33806,10,2,0.65,157,...,368,37,12,7,0.388554,0.428222,0.680332,0.550856,49.256949,36.0
66,9653,2020-11-23,1994-05-23,P,Malang,114910,106,41,0.32,615,...,1794,1224,120,11,0.906105,0.923178,0.406694,0.792226,81.488049,14.0


In [62]:
df.to_clipboard()

In [65]:
df_train_new = df[['Total Voucher/Discount (in scale)', 'GMV (in scale)', 'Count of Active Days', 'Count of Cancelled Orders','Profit (in scale)', 'Commission (in scale)', 'Count of SKU', 'Count of Category', 'score']]
df_train_new

,Total Voucher/Discount (in scale),GMV (in scale),Count of Active Days,Count of Cancelled Orders,Profit (in scale),Commission (in scale),Count of SKU,Count of Category,score
0,0,20760,6,0,104,40,9,5,49.161281
1,55,10335,14,0,236,158,57,10,43.466049
2,132,58894,22,4,776,625,90,11,67.604277
3,623,550416,30,6,6640,3188,214,13,96.070839
4,72,34808,19,0,502,616,63,10,58.521557
...,...,...,...,...,...,...,...,...,...
63,0,2121,4,1,30,6,9,3,40.294276
64,0,6503,6,0,92,33,30,5,40.994797
65,157,33806,10,0,368,37,12,7,49.256949
66,615,114910,27,3,1794,1224,120,11,81.488049


In [66]:
df_train_new.corr(method='pearson')

,Total Voucher/Discount (in scale),GMV (in scale),Count of Active Days,Count of Cancelled Orders,Profit (in scale),Commission (in scale),Count of SKU,Count of Category,score
Total Voucher/Discount (in scale),1.000000,0.619722,0.584249,0.331039,0.857552,0.832805,0.612883,0.383851,0.635112
GMV (in scale),0.619722,1.000000,0.664256,0.437203,0.765526,0.792037,0.686806,0.454222,0.801486
Count of Active Days,0.584249,0.664256,1.000000,0.606957,0.679040,0.672073,0.875978,0.807498,0.921620
Count of Cancelled Orders,0.331039,0.437203,0.606957,1.000000,0.521396,0.514427,0.638526,0.487728,0.635535
Profit (in scale),0.857552,0.765526,0.679040,0.521396,1.000000,0.977227,0.817387,0.502146,0.782838
Commission (in scale),0.832805,0.792037,0.672073,0.514427,0.977227,1.000000,0.819223,0.485411,0.782555
Count of SKU,0.612883,0.686806,0.875978,0.638526,0.817387,0.819223,1.000000,0.759441,0.859936
Count of Category,0.383851,0.454222,0.807498,0.487728,0.502146,0.485411,0.759441,1.000000,0.686892
score,0.635112,0.801486,0.921620,0.635535,0.782838,0.782555,0.859936,0.686892,1.000000


In [67]:
df_train_new.corr(method='spearman')

,Total Voucher/Discount (in scale),GMV (in scale),Count of Active Days,Count of Cancelled Orders,Profit (in scale),Commission (in scale),Count of SKU,Count of Category,score
Total Voucher/Discount (in scale),1.000000,0.678849,0.750312,0.537409,0.698616,0.754871,0.676527,0.653736,0.686173
GMV (in scale),0.678849,1.000000,0.901673,0.634895,0.943543,0.857148,0.820090,0.764763,0.957209
Count of Active Days,0.750312,0.901673,1.000000,0.675552,0.934124,0.862317,0.927083,0.866239,0.907735
Count of Cancelled Orders,0.537409,0.634895,0.675552,1.000000,0.688875,0.735873,0.690640,0.600483,0.652769
Profit (in scale),0.698616,0.943543,0.934124,0.688875,1.000000,0.914087,0.909658,0.817646,0.914952
Commission (in scale),0.754871,0.857148,0.862317,0.735873,0.914087,1.000000,0.859137,0.766977,0.826475
Count of SKU,0.676527,0.820090,0.927083,0.690640,0.909658,0.859137,1.000000,0.881922,0.834732
Count of Category,0.653736,0.764763,0.866239,0.600483,0.817646,0.766977,0.881922,1.000000,0.771362
score,0.686173,0.957209,0.907735,0.652769,0.914952,0.826475,0.834732,0.771362,1.000000
